In [7]:
## install required packages
!pip install swig
!pip install wrds
!pip install pyportfolioopt
# ## install finrl library
!pip install -q condacolab
import condacolab
condacolab.install()
!apt-get update -y -qq && apt-get install -y -qq cmake libopenmpi-dev python3-dev zlib1g-dev libgl1-mesa-glx swig
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
  File "/tmp/ipykernel_6083/4260978696.py", line 7, in <module>
    import condacolab
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/condacolab.py", line 22, in <module>
    from distutils.spawn import find_executable
ModuleNotFoundError: No module named 'distutils'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", lin

In [6]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

%matplotlib inline
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl.meta.data_processor import DataProcessor

from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint

import sys
sys.path.append("../FinRL")

import itertools

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3553, in run_code
  File "/tmp/ipykernel_6083/4094496222.py", line 1, in <module>
    import pandas as pd
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/pandas/__init__.py", line 19, in <module>
    raise ImportError(
ImportError: Unable to import required dependencies:
numpy: No module named 'secrets'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/pygments/styles/__init__.py", line 45, in get_style_by_name
ModuleNotFoundError: No module named 'pygments.styles.default'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/azgor/MLProject/Stock-Trader/.conda/lib/python3.11/site-packages/IPython/core/intera

In [ ]:
from finrl import config
from finrl import config_tickers
import os
from finrl.main import check_and_make_directories
from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

In [ ]:
# import and Create new folder
import os
from google.colab import files
import shutil

new_folder = 'data'

if os.path.isdir(new_folder):
  shutil.rmtree(new_folder)

os.mkdir(new_folder)

# Upload Files
uploaded = files.upload()
for filename in uploaded.keys():
  dst_path = os.path.join(new_folder, filename)
  print(f'move {filename} to {dst_path}')
  shutil.move(filename, dst_path)

Saving output-2009-1-01-to-2009-1-31.csv to output-2009-1-01-to-2009-1-31.csv
move output-2009-1-01-to-2009-1-31.csv to data/output-2009-1-01-to-2009-1-31.csv


In [ ]:
df = pd.read_csv('./data/output-2009-1-01-to-2009-1-31.csv', index_col=0)
df.drop(['#', 'LTP*', 'YCP', 'TRADE', 'VALUE (mn)'], axis=1, inplace=True)
df.rename(columns={'DATE': 'date', 'TRADING CODE': 'tic', 'HIGH': 'high', 'LOW': 'low', 'OPENP*': 'open', 'CLOSEP*': 'close', 'VOLUME': 'volume'}, inplace=True)
df.sort_values(['date','tic'],ignore_index=True).head()

,date,tic,high,low,open,close,volume
0,2009-01-01,1STBSRS,890,830,847.5,875,"8,200"
1,2009-01-01,1STICB,0,0,"5,000","5,000",0
2,2009-01-01,2NDICB,"2,140","2,000","2,140","2,035",20
3,2009-01-01,3RDICB,"1,048","1,030","1,045","1,040.25",190
4,2009-01-01,4THICB,"1,039.75","1,002","1,003","1,024",50


In [ ]:
df['high'] = df['high'].str.replace(',', '').astype(float)
df['low'] = df['low'].str.replace(',', '').astype(float)
df['open'] = df['open'].str.replace(',', '').astype(float)
df['close'] = df['close'].str.replace(',', '').astype(float)
df['volume'] = df['volume'].str.replace(',', '').astype(float)

# df.applymap(lambda x: float(x.replace(',', '')) if isinstance(x, str) else x)

In [ ]:
# prompt: drop those rows which has 0 as values

df = df.replace(0, np.nan).dropna()


In [ ]:
# prompt: count the number of row with  0 as values

zero_count = df.isin([0]).sum().sum()
print(f"Number of rows with 0 values: {zero_count}")


Number of rows with 0 values: 0


In [ ]:
# prompt: there is a column name 'date' which types is string. Now add another column to the df which contains the day of the week.

df['day'] = pd.to_datetime(df['date']).dt.dayofweek


In [ ]:
df['short_resistance']= df['high'].rolling(window=10,min_periods=0).max()
df['short_support']= df['low'].rolling(window=10,min_periods=0).min()
df['long_resistance']= df['high'].rolling(window=50,min_periods=0).max()
df['long_support']= df['low'].rolling(window=50,min_periods=0).min()
df = df.reset_index(drop=True)
df = df.sort_values(by=['date','tic']).reset_index(drop=True)

In [ ]:
df.head()

,date,tic,high,low,open,close,volume,day,short_resistance,short_support,long_resistance,long_support
0,2009-01-01,1STBSRS,890.00,830.0,847.5,875.00,8200.0,3,890.0,830.0,890.0,830.0
1,2009-01-01,2NDICB,2140.00,2000.0,2140.0,2035.00,20.0,3,2140.0,830.0,2140.0,830.0
2,2009-01-01,3RDICB,1048.00,1030.0,1045.0,1040.25,190.0,3,2140.0,830.0,2140.0,830.0
3,2009-01-01,4THICB,1039.75,1002.0,1003.0,1024.00,50.0,3,2140.0,830.0,2140.0,830.0
4,2009-01-01,5THICB,1049.00,1002.0,1049.0,1021.75,410.0,3,2140.0,830.0,2140.0,830.0


In [ ]:
df.shape

(4964, 12)

In [ ]:
df = df.dropna()
df.shape

(4964, 12)

In [ ]:
INDICATORS

['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [ ]:
data = df.copy()
df_price_pivot = data.pivot(index="date", columns="tic", values="close")
df_price_pivot = df_price_pivot.pct_change()
unique_date = data.date.unique()
# start after a year
start = 252
turbulence_index = [0] * start
# turbulence_index = [0]
count = 0
for i in range(start, len(unique_date)):
  current_price = df_price_pivot[df_price_pivot.index == unique_date[i]]
  # use one year rolling window to calcualte covariance
  hist_price = df_price_pivot[
      (df_price_pivot.index < unique_date[i])
      & (df_price_pivot.index >= unique_date[i - 252])
  ]
  # Drop tickers which has number missing values more than the "oldest" ticker
  filtered_hist_price = hist_price.iloc[
      hist_price.isna().sum().min() :
  ].dropna(axis=1)

  cov_temp = filtered_hist_price.cov()
  current_temp = current_price[[x for x in filtered_hist_price]] - np.mean(
      filtered_hist_price, axis=0
  )
  # cov_temp = hist_price.cov()
  # current_temp=(current_price - np.mean(hist_price,axis=0))

  temp = current_temp.values.dot(np.linalg.pinv(cov_temp)).dot(
      current_temp.values.T
  )
  if temp > 0:
      count += 1
      if count > 2:
          turbulence_temp = temp[0][0]
      else:
          # avoid large outlier because of the calculation just begins
          turbulence_temp = 0
  else:
      turbulence_temp = 0
  turbulence_index.append(turbulence_temp)
print(df_price_pivot.index)
print(len(turbulence_index))
try:
  turbulence_index = pd.DataFrame(
      {"date": df_price_pivot.index, "turbulence": turbulence_index}
  )
except ValueError:
  raise Exception("Turbulence information could not be added.")

Index(['2009-01-01', '2009-01-04', '2009-01-05', '2009-01-06', '2009-01-07',
       '2009-01-11', '2009-01-12', '2009-01-13', '2009-01-14', '2009-01-15',
       '2009-01-18', '2009-01-19', '2009-01-20', '2009-01-21', '2009-01-22',
       '2009-01-25', '2009-01-26', '2009-01-27', '2009-01-28', '2009-01-29'],
      dtype='object', name='date')
252


<ipython-input-57-8637ca15705a>:3: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  df_price_pivot = df_price_pivot.pct_change()


Exception: Turbulence information could not be added.

In [ ]:
fe = FeatureEngineer(
                    use_technical_indicator=True,
                    tech_indicator_list = INDICATORS,
                    use_vix=True,
                    use_turbulence=False,
                    user_defined_feature = False)
# turbulance is giving error
df = df.fillna(value = 0)
processed = fe.preprocess_data(df)

/usr/local/lib/python3.10/site-packages/yfinance/utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed

Successfully added technical indicators
Shape of DataFrame:  (18, 8)
Successfully added vix


In [ ]:
# prompt: add a column name 'turbulence' with type float and value 0

processed['turbulence'] = 0.0
processed['turbulence'] = processed['turbulence'].astype('float')


In [ ]:
list_ticker = processed["tic"].unique().tolist()
list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
combination = list(itertools.product(list_date,list_ticker))

processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
processed_full = processed_full[processed_full['date'].isin(processed['date'])]
processed_full = processed_full.sort_values(['date','tic'])

processed_full = processed_full.fillna(0)

In [ ]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)

,date,tic,high,low,open,close,volume,day,short_resistance,short_support,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-05,1STBSRS,926.0,851.0,926.00,860.50,9000.0,0.0,3370.0,19.0,...,-0.583468,962.194361,814.972306,43.365586,-27.826087,9.382367,888.583333,888.583333,39.080002,0.0
1,2009-01-05,6THICB,640.0,578.0,640.00,581.25,4060.0,0.0,5212.0,19.0,...,1.462351,651.466686,492.033314,75.287069,68.101761,100.000000,571.750000,571.750000,39.080002,0.0
2,2009-01-05,8THICB,599.0,522.0,599.00,555.75,4500.0,0.0,5212.0,19.0,...,1.857242,618.703483,455.129850,88.685015,66.837716,100.000000,536.916667,536.916667,39.080002,0.0
3,2009-01-05,ABBANK,800.0,793.5,793.50,795.25,40675.0,0.0,5212.0,522.0,...,-0.318218,811.130710,789.535957,0.000000,-94.428152,100.000000,800.333333,800.333333,39.080002,0.0
4,2009-01-05,ACI,536.0,520.1,536.00,524.80,118550.0,0.0,5212.0,520.1,...,-0.046853,530.647516,522.485817,39.485628,-58.267717,3.333333,526.566667,526.566667,39.080002,0.0
5,2009-01-05,ACIFORMULA,222.0,212.6,217.00,217.20,894500.0,0.0,1991.0,212.6,...,0.235969,221.616237,205.450430,100.000000,70.886076,100.000000,213.533333,213.533333,39.080002,0.0
6,2009-01-05,AFTABAUTO,373.0,355.5,368.00,362.00,19390.0,0.0,1199.0,212.6,...,0.203995,376.836111,346.830556,66.666667,39.080460,100.000000,361.833333,361.833333,39.080002,0.0
7,2009-01-05,AGNISYSL,68.4,62.0,67.00,64.40,448500.0,0.0,1199.0,62.0,...,0.073796,68.134800,59.798534,73.896830,47.058824,100.000000,63.966667,63.966667,39.080002,0.0
8,2009-01-05,AGRANINS,319.0,305.0,310.00,307.75,32900.0,0.0,1199.0,62.0,...,0.331062,319.357658,289.642342,86.920981,47.289157,100.000000,304.500000,304.500000,39.080002,0.0
9,2009-01-05,AIMS1STMF,17.2,15.8,17.09,15.89,6885000.0,0.0,800.0,15.8,...,0.008320,17.594729,14.631938,56.831575,26.826347,76.288245,16.113333,16.113333,39.080002,0.0


In [ ]:
mvo_df = processed_full.sort_values(['date','tic'],ignore_index=True)[['date','tic','close']]

In [ ]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2009-01-25'
TRADE_START_DATE = '2009-01-26'
TRADE_END_DATE = '2009-01-31'
train = data_split(processed_full, TRAIN_START_DATE,TRAIN_END_DATE)
trade = data_split(processed_full, TRADE_START_DATE,TRADE_END_DATE)
print(len(train))
print(len(trade))

1910
573


In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 191, State Space: 1911


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [ ]:
agent = DRLAgent(env = env_train)

if_using_a2c = True
if_using_ddpg = False
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

### Agent 1: A2C


In [ ]:
agent = DRLAgent(env = env_train)
model_a2c = agent.get_model("a2c")

if if_using_a2c:
  # set up logger
  tmp_path = RESULTS_DIR + '/a2c'
  new_logger_a2c = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_a2c.set_logger(new_logger_a2c)


{'n_steps': 5, 'ent_coef': 0.01, 'learning_rate': 0.0007}
Using cpu device
Logging to results/a2c


In [ ]:
trained_a2c = agent.train_model(model=model_a2c,
                             tb_log_name='a2c',
                             total_timesteps=50000) if if_using_a2c else None

Streaming output truncated to the last 5000 lines.
-------------------------------------
| time/                 |           |
|    fps                | 281       |
|    iterations         | 1100      |
|    time_elapsed       | 19        |
|    total_timesteps    | 5500      |
| train/                |           |
|    entropy_loss       | -274      |
|    explained_variance | 0.718     |
|    learning_rate      | 0.0007    |
|    n_updates          | 1099      |
|    policy_loss        | 188       |
|    reward             | 1.1019584 |
|    std                | 1.02      |
|    value_loss         | 0.651     |
-------------------------------------
day: 9, episode: 560
begin_total_asset: 1000000.00
end_total_asset: 987937.93
total_reward: -12062.07
total_cost: 2449.40
total_trades: 977
Sharpe: -1.241
day: 9, episode: 570
begin_total_asset: 1000000.00
end_total_asset: 962968.86
total_reward: -37031.14
total_cost: 1386.58
total_trades: 973
Sharpe: -4.452
day: 9, episode: 580
begin_tota

### Agent 2: DDPG

In [ ]:
agent = DRLAgent(env = env_train)
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
  # set up logger
  tmp_path = RESULTS_DIR + '/ddpg'
  new_logger_ddpg = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ddpg.set_logger(new_logger_ddpg)

{'batch_size': 128, 'buffer_size': 50000, 'learning_rate': 0.001}
Using cpu device


In [ ]:
trained_ddpg = agent.train_model(model=model_ddpg,
                             tb_log_name='ddpg',
                             total_timesteps=50000) if if_using_ddpg else None

### Agent 3: PPO

In [ ]:
agent = DRLAgent(env = env_train)
PPO_PARAMS = {
    "n_steps": 2048,
    "ent_coef": 0.01,
    "learning_rate": 0.00025,
    "batch_size": 128,
}
model_ppo = agent.get_model("ppo",model_kwargs = PPO_PARAMS)

if if_using_ppo:
  # set up logger
  tmp_path = RESULTS_DIR + '/ppo'
  new_logger_ppo = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_ppo.set_logger(new_logger_ppo)

{'n_steps': 2048, 'ent_coef': 0.01, 'learning_rate': 0.00025, 'batch_size': 128}
Using cpu device


In [ ]:
trained_ppo = agent.train_model(model=model_ppo,
                             tb_log_name='ppo',
                             total_timesteps=50000) if if_using_ppo else None

### Agent 4: TD3

In [ ]:
agent = DRLAgent(env = env_train)
TD3_PARAMS = {"batch_size": 100,
              "buffer_size": 1000000,
              "learning_rate": 0.001}

model_td3 = agent.get_model("td3",model_kwargs = TD3_PARAMS)

if if_using_td3:
  # set up logger
  tmp_path = RESULTS_DIR + '/td3'
  new_logger_td3 = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_td3.set_logger(new_logger_td3)

{'batch_size': 100, 'buffer_size': 1000000, 'learning_rate': 0.001}
Using cpu device


/usr/local/lib/python3.10/site-packages/stable_baselines3/common/buffers.py:241: UserWarning: This system does not have apparently enough memory to store the complete replay buffer 34.79GB > 4.69GB
  warnings.warn(


In [ ]:
trained_td3 = agent.train_model(model=model_td3,
                             tb_log_name='td3',
                             total_timesteps=50000) if if_using_td3 else None

### Agent 5: SAC

In [ ]:
agent = DRLAgent(env = env_train)
SAC_PARAMS = {
    "batch_size": 128,
    "buffer_size": 100000,
    "learning_rate": 0.0001,
    "learning_starts": 100,
    "ent_coef": "auto_0.1",
}

model_sac = agent.get_model("sac",model_kwargs = SAC_PARAMS)

if if_using_sac:
  # set up logger
  tmp_path = RESULTS_DIR + '/sac'
  new_logger_sac = configure(tmp_path, ["stdout", "csv", "tensorboard"])
  # Set new logger
  model_sac.set_logger(new_logger_sac)

{'batch_size': 128, 'buffer_size': 100000, 'learning_rate': 0.0001, 'learning_starts': 100, 'ent_coef': 'auto_0.1'}
Using cpu device


In [ ]:
trained_sac = agent.train_model(model=model_sac,
                             tb_log_name='sac',
                             total_timesteps=50000) if if_using_sac else None

## In-sample Performance

Assume that the initial capital is $1,000,000.

### Set turbulence threshold
Set the turbulence threshold to be greater than the maximum of insample turbulence data. If current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [ ]:
data_risk_indicator = processed_full[(processed_full.date<TRAIN_END_DATE) & (processed_full.date>=TRAIN_START_DATE)]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])

In [ ]:
insample_risk_indicator.vix.describe()

count    10.000000
mean     46.064000
std       5.452953
min      38.560001
25%      43.300000
50%      46.129999
75%      48.677500
max      56.650002
Name: vix, dtype: float64

In [ ]:
insample_risk_indicator.vix.quantile(0.996)

56.446601470947265

In [ ]:
insample_risk_indicator.turbulence.describe()

count    10.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
Name: turbulence, dtype: float64

In [ ]:
insample_risk_indicator.turbulence.quantile(0.996)

0.0

### Trading (Out-of-sample Performance)

We update periodically in order to take full advantage of the data, e.g., retrain quarterly, monthly or weekly. We also tune the parameters along the way, in this notebook we use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends.

Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [ ]:
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()

In [ ]:
trade.head()

,date,tic,high,low,open,close,volume,day,short_resistance,short_support,...,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2009-01-26,1STBSRS,810.0,786.00,789.0,798.75,3150.0,0.0,3125.0,21.90,...,-11.501742,912.699459,729.977012,39.264153,-49.118972,16.470279,821.338235,821.338235,45.689999,0.0
0,2009-01-26,6THICB,548.0,526.25,530.0,534.75,1000.0,0.0,5200.0,21.90,...,-5.838398,597.005605,500.759100,49.578484,-46.809854,17.728077,548.882353,548.882353,45.689999,0.0
0,2009-01-26,8THICB,530.0,502.25,510.0,513.00,1450.0,0.0,5200.0,21.90,...,-6.106203,574.432304,460.744166,53.257310,-3.253979,33.913895,517.588235,517.588235,45.689999,0.0
0,2009-01-26,ABBANK,740.0,718.00,740.0,720.50,37035.0,0.0,5200.0,21.90,...,-8.771555,816.841769,715.011172,16.461563,-137.231601,47.094202,765.926471,765.926471,45.689999,0.0
0,2009-01-26,ACI,541.0,515.00,538.5,516.50,111250.0,0.0,5200.0,502.25,...,0.905433,536.333541,481.984106,49.108051,83.423100,28.728688,509.158824,509.158824,45.689999,0.0


In [ ]:
trained_moedl = trained_a2c
df_account_value_a2c, df_actions_a2c = DRLAgent.DRL_prediction(
    model=trained_moedl,
    environment = e_trade_gym)

hit end!


In [ ]:
trained_moedl = trained_ddpg
df_account_value_ddpg, df_actions_ddpg = DRLAgent.DRL_prediction(
    model=trained_moedl,
    environment = e_trade_gym)

AttributeError: 'NoneType' object has no attribute 'predict'

In [ ]:
trained_moedl = trained_ppo
df_account_value_ppo, df_actions_ppo = DRLAgent.DRL_prediction(
    model=trained_moedl,
    environment = e_trade_gym)

In [ ]:
trained_moedl = trained_td3
df_account_value_td3, df_actions_td3 = DRLAgent.DRL_prediction(
    model=trained_moedl,
    environment = e_trade_gym)

In [ ]:
trained_moedl = trained_sac
df_account_value_sac, df_actions_sac = DRLAgent.DRL_prediction(
    model=trained_moedl,
    environment = e_trade_gym)

In [ ]:
df_account_value_a2c

,date,account_value
0,2009-01-26,1000000.00000
1,2009-01-27,982649.50705
2,2009-01-28,994245.65155


In [ ]:
df_actions_a2c

,1STBSRS,6THICB,8THICB,ABBANK,ACI,ACIFORMULA,AFTABAUTO,AGNISYSL,AGRANINS,AIMS1STMF,...,TITASGAS,TRUSTBANK,ULC,UNIONCAP,UNITEDINS,USMANIAGL,UTTARABANK,UTTARAFIN,WONDERTOYS,ZEALBANGLA
date,,,,,,,,,,,,,,,,,,,,,
2009-01-26,0,0,0,0,0,0,0,0,0,100,...,0,0,0,0,0,0,0,0,0,0
2009-01-27,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<a id='7'></a>
# Part 6.5: Mean Variance Optimization

In [ ]:
mvo_df.head()

,date,tic,close
0,2009-01-05,1STBSRS,860.50
1,2009-01-05,6THICB,581.25
2,2009-01-05,8THICB,555.75
3,2009-01-05,ABBANK,795.25
4,2009-01-05,ACI,524.80


In [ ]:
fst = mvo_df
fst = fst.iloc[0*29:0*29+29, :]
tic = fst['tic'].tolist()

mvo = pd.DataFrame()

for k in range(len(tic)):
  mvo[tic[k]] = 0

for i in range(mvo_df.shape[0]//29):
  n = mvo_df
  n = n.iloc[i*29:i*29+29, :]
  date = n['date'][i*29]
  mvo.loc[date] = n['close'].tolist()

In [ ]:
mvo.shape[0]

13

### Helper functions

In [ ]:
from scipy import optimize
from scipy.optimize import linprog

#function obtains maximal return portfolio using linear programming

def MaximizeReturns(MeanReturns, PortfolioSize):

  #dependencies


  c = (np.multiply(-1, MeanReturns))
  A = np.ones([PortfolioSize,1]).T
  b=[1]
  res = linprog(c, A_ub = A, b_ub = b, bounds = (0,1), method = 'simplex')

  return res

def MinimizeRisk(CovarReturns, PortfolioSize):

  def f(x, CovarReturns):
    func = np.matmul(np.matmul(x, CovarReturns), x.T)
    return func

  def constraintEq(x):
    A=np.ones(x.shape)
    b=1
    constraintVal = np.matmul(A,x.T)-b
    return constraintVal

  xinit=np.repeat(0.1, PortfolioSize)
  cons = ({'type': 'eq', 'fun':constraintEq})
  lb = 0
  ub = 1
  bnds = tuple([(lb,ub) for x in xinit])

  opt = optimize.minimize (f, x0 = xinit, args = (CovarReturns),  bounds = bnds, \
                             constraints = cons, tol = 10**-3)

  return opt

def MinimizeRiskConstr(MeanReturns, CovarReturns, PortfolioSize, R):

  def  f(x,CovarReturns):

    func = np.matmul(np.matmul(x,CovarReturns ), x.T)
    return func

  def constraintEq(x):
    AEq=np.ones(x.shape)
    bEq=1
    EqconstraintVal = np.matmul(AEq,x.T)-bEq
    return EqconstraintVal

  def constraintIneq(x, MeanReturns, R):
    AIneq = np.array(MeanReturns)
    bIneq = R
    IneqconstraintVal = np.matmul(AIneq,x.T) - bIneq
    return IneqconstraintVal


  xinit=np.repeat(0.1, PortfolioSize)
  cons = ({'type': 'eq', 'fun':constraintEq},
          {'type':'ineq', 'fun':constraintIneq, 'args':(MeanReturns,R) })
  lb = 0
  ub = 1
  bnds = tuple([(lb,ub) for x in xinit])

  opt = optimize.minimize (f, args = (CovarReturns), method ='trust-constr',  \
                x0 = xinit,   bounds = bnds, constraints = cons, tol = 10**-3)

  return opt

In [ ]:
def StockReturnsComputing(StockPrice, Rows, Columns):
  import numpy as np
  StockReturn = np.zeros([Rows-1, Columns])
  for j in range(Columns):        # j: Assets
    for i in range(Rows-1):     # i: Daily Prices
      StockReturn[i,j]=((StockPrice[i+1, j]-StockPrice[i,j])/StockPrice[i,j])* 100

  return StockReturn

### Calculate mean returns and variance-covariance matrix

In [ ]:
# Obtain optimal portfolio sets that maximize return and minimize risk

#Dependencies
import numpy as np
import pandas as pd


#input k-portfolio 1 dataset comprising 15 stocks
# StockFileName = './DJIA_Apr112014_Apr112019_kpf1.csv'

Rows = 1259  #excluding header
Columns = 15  #excluding date
portfolioSize = 29 #set portfolio size

#read stock prices in a dataframe
# df = pd.read_csv(StockFileName,  nrows= Rows)

#extract asset labels
# assetLabels = df.columns[1:Columns+1].tolist()
# print(assetLabels)

#extract asset prices
# StockData = df.iloc[0:, 1:]
StockData = mvo.head(mvo.shape[0]-336)
TradeData = mvo.tail(336)
# df.head()
TradeData.to_numpy()


array([[ 117.2 , 3110.  ,  159.75,  219.  , 1005.  ,  236.  ,   65.  ,
         510.  ,  419.5 ,  763.25,  119.5 , 1763.  , 2014.75, 3228.  ,
         770.25,   40.  ,   22.  ,  843.5 ,  565.5 ,  532.5 ,  784.75,
         504.7 ,  207.6 ,  354.5 ,   61.5 ,  298.25,   15.34,  423.  ,
          68.25],
       [1991.  , 3161.25,  744.  ,   41.25,   22.4 ,  839.5 ,  564.75,
         531.25,  783.75,  505.5 ,  205.5 ,  362.  ,   62.2 ,  305.25,
          15.56,  418.25,   68.75,  128.6 , 1102.75,   60.5 , 2323.25,
         909.25,  469.5 ,  932.25,  281.1 ,  186.25,    8.8 ,  224.75,
         298.8 ],
       [  42.4 ,  207.  ,  149.5 ,  306.75,  100.  ,  115.1 , 3062.5 ,
         163.5 ,  215.  , 1079.5 ,  228.  ,   66.5 ,  493.75,  415.75,
         760.  ,  115.  , 1804.  , 1981.75, 3185.25,  735.25,   40.25,
          21.3 ,  840.25,  567.75,  544.25,  776.  ,  505.5 ,  212.9 ,
         346.25],
       [ 502.  ,  407.75,  729.75,  114.9 , 1663.5 , 1907.75, 3106.75,
         728.75,   40.7

In [ ]:
#compute asset returns
arStockPrices = np.asarray(StockData)
[Rows, Cols]=arStockPrices.shape
arReturns = StockReturnsComputing(arStockPrices, Rows, Cols)


#compute mean returns and variance covariance matrix of returns
meanReturns = np.mean(arReturns, axis = 0)
covReturns = np.cov(arReturns, rowvar=False)

#set precision for printing results
np.set_printoptions(precision=3, suppress = True)

#display mean returns and variance-covariance matrix of returns
print('Mean returns of assets in k-portfolio 1\n', meanReturns)
print('Variance-Covariance matrix of returns\n', covReturns)


ValueError: negative dimensions are not allowed

In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier

ef_mean = EfficientFrontier(meanReturns, covReturns, weight_bounds=(0, 0.5))
raw_weights_mean = ef_mean.max_sharpe()
cleaned_weights_mean = ef_mean.clean_weights()
mvo_weights = np.array([1000000 * cleaned_weights_mean[i] for i in range(29)])
mvo_weights

In [ ]:
StockData.tail(1)

In [ ]:
LastPrice = np.array([1/p for p in StockData.tail(1).to_numpy()[0]])
Initial_Portfolio = np.multiply(mvo_weights, LastPrice)
Initial_Portfolio

In [ ]:
Portfolio_Assets = TradeData @ Initial_Portfolio
MVO_result = pd.DataFrame(Portfolio_Assets, columns=["Mean Var"])
MVO_result

<a id='6'></a>
# Part 7: Backtesting Results
Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

In [ ]:
df_result_a2c = df_account_value_a2c.set_index(df_account_value_a2c.columns[0])
df_result_ddpg = df_account_value_ddpg.set_index(df_account_value_ddpg.columns[0])
df_result_td3 = df_account_value_td3.set_index(df_account_value_td3.columns[0])
df_result_ppo = df_account_value_ppo.set_index(df_account_value_ppo.columns[0])
df_result_sac = df_account_value_sac.set_index(df_account_value_sac.columns[0])
df_account_value_a2c.to_csv("df_account_value_a2c.csv")
#baseline stats
print("==============Get Baseline Stats===========")
df_dji_ = get_baseline(
        ticker="^DJI",
        start = TRADE_START_DATE,
        end = TRADE_END_DATE)
stats = backtest_stats(df_dji_, value_col_name = 'close')
df_dji = pd.DataFrame()
df_dji['date'] = df_account_value_a2c['date']
df_dji['account_value'] = df_dji_['close'] / df_dji_['close'][0] * env_kwargs["initial_amount"]
df_dji.to_csv("df_dji.csv")
df_dji = df_dji.set_index(df_dji.columns[0])
df_dji.to_csv("df_dji+.csv")

result = pd.merge(df_result_a2c, df_result_ddpg, left_index=True, right_index=True, suffixes=('_a2c', '_ddpg'))
result = pd.merge(result, df_result_td3, left_index=True, right_index=True, suffixes=('', '_td3'))
result = pd.merge(result, df_result_ppo, left_index=True, right_index=True, suffixes=('', '_ppo'))
result = pd.merge(result, df_result_sac, left_index=True, right_index=True, suffixes=('', '_sac'))
result = pd.merge(result, MVO_result, left_index=True, right_index=True, suffixes=('', '_mvo'))
result = pd.merge(result, df_dji, left_index=True, right_index=True, suffixes=('', '_dji'))
result.columns = ['a2c', 'ddpg', 'td3', 'ppo', 'sac', 'mean var', 'dji']

print("result: ", result)
result.to_csv("result.csv")

In [ ]:
%matplotlib inline
plt.rcParams["figure.figsize"] = (15,5)
plt.figure();
result.plot();